# Plant Growth Module (PGM) - DFS2 Map Generator

This notebook generates spatially distributed DFS2 maps for ECO Lab Plant Growth Module parameters based on land use classification and species-specific values.

## Workflow Overview
1. **Configure Paths**: Define input files and output directory
2. **Validate**: Check all required files exist
3. **Load Land Use Data**: Read spatial DFS2 grid and classification mapping
4. **Process Templates**: Read parameter templates and generate DFS2 maps for each parameter where type=1
5. **Verify**: Summarize generated files and validate output geometry


In [ ]:
from pathlib import Path
import warnings

# Add the src directory to the Python path so pgm_helper can be imported directly

# Import all required libraries
import pandas as pd
import numpy as np
import mikeio
import os
from src.pgm_helper import (
    VAL_COLS,
    CLASS_COLS,
    ID_COLS,
    VALUE_COLS,
    KEY_COLS,
    find_col,
    confirm_columns,
    generate_dfs2_map,
    validate_paths,
)

warnings.filterwarnings("ignore")

## 📋 Step 0: Configuration

**Edit the paths below to match your data location.**

This cell defines:
- Input DFS2 file with land use spatial data
- Land use classification template (CODE → CLASS mapping)
- Parameter templates (Constants and Initial Conditions)
- Output directory for generated DFS2 files
- Processing options (AUTO_CONFIRM for batch mode)


In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# 📥 INPUT FILES (ABSOLUTE PATHS)
# ═══════════════════════════════════════════════════════════════════════════
# Edit these paths as needed - use absolute paths only

# Land use spatial data (DFS2 format)
LANDUSE_DFS2 = Path(
    r""
)

# Land use classification mapping (CSV with CODE and CLASS columns)
LU_TEMPLATE = Path(
    r""
)

# Template files for generating maps (CSV files)
TEMPLATE_FILES = [
    Path(
        r""
    ),
    Path(
        r""
    ),
]

# ═══════════════════════════════════════════════════════════════════════════
# 📤 OUTPUT DIRECTORY (ABSOLUTE PATH)
# ═══════════════════════════════════════════════════════════════════════════
OUTPUT_DIR = Path(
    r""
)

AUTO_CONFIRM = False


# Validate all paths
errors = validate_paths(LANDUSE_DFS2, LU_TEMPLATE, TEMPLATE_FILES, OUTPUT_DIR)

if errors:
    raise FileNotFoundError("Fix the errors above and re-run this cell")

## 📍 Step 1: Load Land Use Data

**Read spatial land use grid and create code-to-species mapping.**


### Step 1.A: Load Spatial Grid

Read the land use DFS2 file and extract the 2D grid data. Displays:
- Grid dimensions (rows × columns)
- Unique land use codes present in the spatial data


In [ ]:
# Read land use DFS2 file
print("Loading land use DFS2 file...")
landuse_ds = mikeio.Dfs2(LANDUSE_DFS2)
landuse_data = landuse_ds.read()[
    0
].to_numpy()  # Get the first (and typically only) item

print(f"\nLand use grid shape:    {landuse_data.shape}")
print(f"Unique land use codes:  {np.unique(landuse_data)}")

### Step 1.B: Create Code-to-Species Mapping

Read the land use classification template and create a dictionary mapping numeric codes to plant species names.

**Operations:**
- Auto-detects CODE and CLASS columns (case-insensitive)
- Prompts for confirmation (unless AUTO_CONFIRM=True)
- Creates `code_to_species` dictionary used for spatial mapping
- Displays the mapping for verification


In [ ]:
# Read land use classification template
print("\nLoading land use classification...")
lu_df = pd.read_csv(LU_TEMPLATE)

# Identify relevant columns
code_col = find_col(lu_df, VAL_COLS)
class_col = find_col(lu_df, CLASS_COLS)

if code_col is None or class_col is None:
    raise ValueError(
        "Required columns not found in the land use classification template."
    )

# Confirm column detection with user
confirmed = confirm_columns(
    {"Code column": code_col, "Class column": class_col},
    auto_confirm=AUTO_CONFIRM,
    context="Land use classification",
    multi_files=False,
)
if not confirmed:
    raise RuntimeError("User cancelled land use mapping")

# Create dictionary: CODE -> CLASS (speciesID)
code_to_species = dict(zip(lu_df[code_col], lu_df[class_col]))
print("\nCode to Species mapping:")
for code, species in code_to_species.items():
    print(f"  {code:4d} → {species}")

## 🗺️ Step 2: Generate DFS2 Maps

**Process parameter templates and generate spatially distributed DFS2 files.**

For each template (Constants, InitConditions):
- Reads CSV with species-specific parameter values
- Creates DFS2 file for each parameter by mapping values to land use grid
- Generates maps for all parameters defined in the template


### Step 2.A: Process Templates and Generate Maps

**Main processing loop:**

For each CSV template file:
1. Auto-detects columns: speciesID, constant/variable name, value, type
2. Prompts for column confirmation (unless AUTO_CONFIRM=True)
3. For each unique parameter name:
   - Creates species→value mapping
   - Displays mapping (value ← species)
   - Generates DFS2 file by applying values to land use grid
4. Reports number of maps generated per template

**Output:** DFS2 files named `{parameter_name}.dfs2` in OUTPUT_DIR


In [ ]:
# Process all configured templates
total_maps_generated = 0

for template_file in TEMPLATE_FILES:
    print(f"\n{'='*60}")
    print(f"Processing: {template_file.name}")
    print(f"{'='*60}")

    # Read template
    df = pd.read_csv(template_file)

    # Find actual column names in the template
    id_col = find_col(df, ID_COLS)
    value_col = find_col(df, VALUE_COLS)
    key_col = find_col(df, KEY_COLS)

    if not all([id_col, value_col, key_col]):
        print(f"\n⚠ Warning: Missing required columns in {template_file.name}")
        print("  Skipping this template.")
        continue

    # Confirm column detection with user
    confirmed = confirm_columns(
        {
            "Species column": id_col,
            "Value column": value_col,
            "Variable column": key_col,
        },
        auto_confirm=AUTO_CONFIRM,
        context=template_file.name,
    )
    if not confirmed:
        continue

    # Filter only relevant columns
    df = df[[id_col, key_col, value_col]].dropna(subset=[id_col, key_col])

    # Group by key name (constant or variable) and generate maps
    for key_name in df[key_col].unique():
        subset = df[df[key_col] == key_name]
        species_values = dict(zip(subset[id_col], subset[value_col]))

        print(f"\n  Generating map: {key_name}")
        for species, value in species_values.items():
            print(f"    {value:<10} ← {species}")

        # Generate DFS2 map
        output_path = OUTPUT_DIR.joinpath(f"{key_name}.dfs2")
        generate_dfs2_map(
            landuse_data,
            landuse_ds,
            code_to_species,
            species_values,
            output_path,
            key_name,
        )

    maps_count = len(df[key_col].unique())
    print(f"\n✓ Generated {maps_count} maps from {template_file.name}")
    total_maps_generated += maps_count

print(f"\n{'='*60}")
print(f"✓ Total: Generated {total_maps_generated} maps from all templates")

## ✅ Step 3: Summary and Verification

**List all generated files and verify output quality.**

- Lists all `.dfs2` files in output directory with file sizes
- Reads a sample file to verify dimensions and data range
- Confirms geometry matches input land use grid


In [ ]:
# List all generated DFS2 files
print("\n" + "=" * 60)
print("SUMMARY: Generated DFS2 Files")
print("=" * 60)

generated_files = sorted([f for f in os.listdir(OUTPUT_DIR) if f.endswith(".dfs2")])
for i, filename in enumerate(generated_files, 1):
    file_path = OUTPUT_DIR.joinpath(filename)
    file_size = os.path.getsize(file_path) / 1024  # KB
    print(f"{i:2d}. {filename:30s} ({file_size:.1f} KB)")

print(f"\nTotal files generated: {len(generated_files)}")
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# Verification: Read one generated file to check dimensions
if generated_files:
    sample_file = OUTPUT_DIR.joinpath(generated_files[0])
    sample_ds = mikeio.read(sample_file)

    print("\n" + "=" * 60)
    print("VERIFICATION: Sample Output File")
    print("=" * 60)
    print(f"File: {generated_files[0]}")
    print(f"Shape: {sample_ds[0].shape}")
    print(
        f"Data range: [{sample_ds[0].to_numpy().min():.2f}, {sample_ds[0].to_numpy().max():.2f}]"
    )
    print(f"Geometry matches input: {sample_ds.geometry == landuse_ds.geometry}")
    print("\n✓ Script completed successfully!")